# Zestaw 4

### Kamil Burkiewicz

Zadanie dotyczy wykorzystania odległości edycyjnej.

1. Zaimplementuj algorytm obliczania odległości edycyjnej w taki sposób, aby możliwe było określenie przynajmniej jednej sekwencji edycji (dodanie, usunięcie, zmiana znaku), która pozwala w minimalnej liczbie kroków, przekształcić jeden łańcuch w drugi.
2. Na podstawie poprzedniego punktu zaimplementuj prostą wizualizację działania algorytmu, poprzez wskazanie kolejnych wersji pierwszego łańcucha, w których dokonywana jest określona zmiana. "Wizualizacja" może działać w trybie tekstowym. Np. zmiana łańcuch "los" w "kloc" może być zrealizowana następująco:
*k*los (dodanie litery k)
klo*c* (zamiana s->c)
3. Przedstaw wynik działania algorytmu z p. 2 dla następujących par łańcuchów:
  * los - kloc
  * Łódź - Lodz
  * kwintesencja - quintessence
  * ATGAATCTTACCGCCTCG - ATGAGGCTCTGGCCCCTG

4. Zaimplementuj algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów.
5. Korzystając z gotowego tokenizera (np spaCy - https://spacy.io/api/tokenizer) dokonaj podziału załączonego tekstu na tokeny.
6. Stwórz 2 wersje załączonego tekstu, w których usunięto 3% losowych tokenów.
7. Oblicz długość najdłuższego podciągu wspólnych tokenów dla tych tekstów.
8. Korzystając z algorytmu z punktu 4 skonstruuj narzędzie, o działaniu podobnym do narzędzia diff, tzn. wskazującego w dwóch plikach linie, które się różnią. Na wyjściu narzędzia powinny znaleźć się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać skąd dana linia pochodzi (< > - pierwszy/drugi plik) oraz numer linii w danym pliku.
9. Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

#### Imports

In [1]:
import numpy as np
from bisect import bisect
from unidecode import unidecode
import numpy as np
from enum import Enum
from termcolor import colored
from spacy.tokenizer import Tokenizer
from spacy.vocab import Vocab
from spacy.lang.pl import Polish

### Code

#### Z1

In [2]:
def normal_delta(a,b):
    if a == b:
        return 0
    return 1

In [3]:
def edit_distance(x, y, delta):
    edit_table = np.empty((len(x) + 1, len(y) + 1))
    
    for i in range(len(x) + 1):
        edit_table[i, 0] = i
    for j in range(len(y) + 1):
        edit_table[0, j] = j
    for i in range(len(x)):
        k = i + 1
        for j in range(len(y)):
            l = j + 1
            edit_table[k, l] = min(edit_table[k-1, l] + 1,
                                  edit_table[k, l-1] + 1,
                                  edit_table[k-1, l-1] + delta(x[i], y[j]))

    return edit_table

#### Z2

In [4]:
class Operation(Enum):
    EQUAL = 0
    DELETE = 1
    INSERT = 2
    SUBSTITUTE = 3

In [5]:
def seq_of_editions(x, y, edit_table=None):
    if (edit_table is None):
        edit_table = edit_distance(x, y, normal_delta)
    
    i, j = len(x), len(y)
    seq = []
    
    while (i >= 0 and j >= 0):
        if (i == 0 and j == 0):
            break
        if (i > 0 and j > 0):
            # choose any minimal value from left, upper and upper-left cell
            if (edit_table[i - 1][j - 1] <= edit_table[i - 1][j] and
                            edit_table[i - 1][j - 1] <= edit_table[i][j - 1]):
                if (edit_table[i - 1][j - 1] == edit_table[i][j]):
                    i -= 1
                    j -= 1
                    seq += [(Operation.EQUAL, (i, j), (x[i], y[j]))]
                else:
                    i -= 1
                    j -= 1
                    seq += [(Operation.SUBSTITUTE, (i, j), (x[i], y[j]))]
            elif (edit_table[i - 1][j] <= edit_table[i][j - 1] and 
                     edit_table[i - 1][j] <= edit_table[i - 1][j - 1]):
                i -= 1
                seq += [(Operation.DELETE, (i, j), x[i])]
            else:
                j -= 1
                seq += [(Operation.INSERT, (i, j), y[j])]
        elif (i > 0):
            i -= 1
            seq += [(Operation.DELETE, (i, j), x[i])]
        else:
            j -= 1
            seq += [(Operation.INSERT, (i, j), y[j])]
            
    return list(reversed(seq))

In [6]:
def visualize(x, y, editions_sequence=None, verbose=False):
    if (editions_sequence is None):
        editions_sequence = seq_of_editions(x, y)
    
    DELETION_COLOR = "red"
    INSERTION_COLOR = "green"
    SUBSTITITUION_COLOR = "blue"
    WORD_COLOR = "magenta"
    
    if (verbose):
        print(colored(DELETION_COLOR + "\t- deletion", DELETION_COLOR))
        print(colored(INSERTION_COLOR + "\t- insertion", INSERTION_COLOR))
        print(colored(SUBSTITITUION_COLOR + "\t- substitution", SUBSTITITUION_COLOR))
        print(colored("word at the beginning and at the end", WORD_COLOR, attrs=["bold"]), end="\n\n")
        
    word = x
    idx = 0
    
    print(colored(word, color=WORD_COLOR, attrs=["bold"]))
    for operation in editions_sequence:
        letter = operation[2]
        if (operation[0] is Operation.DELETE):
            print(f"{word[:idx]}{colored(letter, DELETION_COLOR, attrs=['reverse'])}{word[idx + 1:]}")
            word = word[:idx] + word[idx + 1:]
            idx -= 1  # Let idx be at the same position after deletion
        elif (operation[0] is Operation.INSERT):
            print(f"{word[:idx]}{colored(letter, INSERTION_COLOR, attrs=['reverse'])}{word[idx:]}")
            word = word[:idx] + letter + word[idx:]
        elif (operation[0] is Operation.SUBSTITUTE):
            # In case of substitution the second element of opertaion is tuple; here only one is needed
            letter = letter[1]
            print(f"{word[:idx]}{colored(letter, SUBSTITITUION_COLOR, attrs=['reverse'])}{word[idx + 1:]}")
            word = word[:idx] + letter + word[idx + 1:]
        idx += 1
    print(colored(word, color=WORD_COLOR, attrs=["bold"]), end="\n\n")

#### Z4

In [7]:
def delta_lcs(x,y):
    if x == y:
        return 0
    else:
        return 2

In [8]:
def lcs(x, y):
    xlen = len(x)
    ylen = len(y)
    return int((xlen + ylen - edit_distance(x, y, delta_lcs)[xlen][ylen]) / 2)

In [9]:
def input_from_file(filename, remove_blank=False):
    """ Read text from file.
    """
    
    with open(filename, "r") as f:
        text = f.read()
    return text

In [10]:
_TOKENIZER_PL = Tokenizer(Polish(Vocab()).vocab)


def tokenize(text, remove_blank=False, punctuation_to_remove=None):
    """  Tokenize given text.
    All sequences of blank characters can be converted to single spaces
    and all punctuation 
    
    
    """

    if (remove_blank):
        text = " ".join(text.split())
    if (punctuation_to_remove):
        for punct in punctuation_to_remove:
            if (len(punct) == 1):
                text = text.replace(punct, '')
    docer = _TOKENIZER_PL(text)
    return list(map(str, docer))

In [11]:
def remove_tokens_randomly(tokens, p=0.5):
    """ Remove random tokens.
    
    Args:
        tokens (list): List of tokens.
        p: Probability of removing a word.
        
    Returns:
        list: List with tokens removed.
    """
    
    new_tokens = []
    for token in tokens:
        if (np.random.uniform() >= p):
            new_tokens += [token]
    
    return new_tokens

#### Z8

In [12]:
def diff(filename1, filename2):
    with open(filename1, "r") as f:
        text1 = f.read()
    with open(filename2, "r") as f:
        text2 = f.read()
    
    tokens1 = [t + '\n' for t in text1.split("\n")]
    tokens2 = [t + '\n' for t in text2.split("\n")]
    
    show_all=False
    if (show_all):
        for i in range(min(len(tokens1), len(tokens2))):
            print(repr(tokens1[i]), repr(tokens2[i]))
            
    edit_table = edit_distance(tokens1, tokens2, delta_lcs)
    changes = seq_of_editions(tokens1, tokens2, edit_table)
    last_change_num = sum(map(lambda op: 1 if (op[0] is not Operation.EQUAL) else 0, changes))
    change_num = 0
    for op in changes:
        #print(op)

        if (op[0] is not Operation.EQUAL):
            line1 = op[1][0]
            line2 = op[1][1]
            if (op[0] is Operation.DELETE):
                print(colored(f"{line1}d{line2}", color="red", attrs=["reverse"]))
                print("<", repr(op[2]))
            if (op[0] is Operation.INSERT):
                print(colored(f"{line1}a{line2}", color="green", attrs=["reverse"]))
                print("<", repr(op[2]))
            if (op[0] is Operation.SUBSTITUTE):
                print(colored(f"{line1}c{line2}", color="blue", attrs=["reverse"]))
                print("<", repr(op[2]))
                print(">", repr(tokens2[line2]))
            change_num += 1
            if (change_num < last_change_num):
                print("-" * 3)

### Tests

In [13]:
x, y = "los", "kloc"
visualize(x, y, verbose=True)
x, y = "Łódź", "Lodz"
visualize(x, y)
x, y = "kwintesencja", "quintessence"
visualize(x, y)
x, y = "ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG"
visualize(x ,y)

red	- deletion
green	- insertion
blue	- substitution
word at the beginning and at the end

los
klos
kloc
kloc

Łódź
Lódź
Lodź
Lodz
Lodz

kwintesencja
qwintesencja
quintesencja
quintessencja
quintessencja
quintessence
quintessence

ATGAATCTTACCGCCTCG
ATGAGTCTTACCGCCTCG
ATGAGGCTTACCGCCTCG
ATGAGGCTCTACCGCCTCG
ATGAGGCTCTGCCGCCTCG
ATGAGGCTCTGGCGCCTCG
ATGAGGCTCTGGCCCCTCG
ATGAGGCTCTGGCCCCTCG
ATGAGGCTCTGGCCCCTG



In [14]:
romeo_and_julia = input_from_file("romeo-i-julia-700.txt")
tokens = tokenize(romeo_and_julia, remove_blank=True, punctuation_to_remove=['.', ','])
random_tokens1 = remove_tokens_randomly(tokens, p=0.03)
random_tokens2 = remove_tokens_randomly(tokens, p=0.03)

In [15]:
print(f"Length of tokens list from original text: {len(tokens)}", end="\n\n")
print(f"Length of longest common subseuquence of tokens: {lcs(random_tokens1, random_tokens2)}")
print(f"\t\t\tVS\nlength of tokens lists of both texts: {len(random_tokens1)} {len(random_tokens2)}")

Length of tokens list from original text: 1888

Length of longest common subseuquence of tokens: 1763
			VS
length of tokens lists of both texts: 1826 1825


In [16]:
# Make valid test files with appropriate new lines 

# Do not remove blank sequences, new lines or punctuation
new_tokens = tokenize(romeo_and_julia)
random_tokens1_2 = remove_tokens_randomly(new_tokens, p=0.03)
random_tokens2_2 = remove_tokens_randomly(new_tokens, p=0.03)

with open("r&j1.txt", "w") as f:
    f.write(" ".join(random_tokens1_2))
with open("r&j2.txt", "w") as f:
    f.write(" ".join(random_tokens2_2))

In [17]:
diff("r&j1.txt", "r&j2.txt")

0c0
< ('Shakespeare \n', 'William Shakespeare \n')
> 'William Shakespeare \n'
---
3c3
< (' tłum. Józef Paszkowski \n', ' tłum. Józef \n')
> ' tłum. Józef \n'
---
10a10
< '  * ESKALUS — książę w Weronie \n'
---
10c11
< ('  * ESKALUS — książę panujący w Weronie \n', '  * PARYS młody Weroneńczyk szlachetnego rodu, krewny księcia \n')
> '  * PARYS młody Weroneńczyk szlachetnego rodu, krewny księcia \n'
---
11c12
< ('  * PARYS — młody Weroneńczyk rodu, krewny księcia \n', '  * MONTEKI, KAPULET — naczelnicy domów nieprzyjaznych sobie \n')
> '  * MONTEKI, KAPULET — naczelnicy domów nieprzyjaznych sobie \n'
---
12c13
< ('  * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie * STARZEC — stryjeczny brat Kapuleta \n', '  * STARZEC — stryjeczny brat Kapuleta \n')
> '  * STARZEC — stryjeczny brat Kapuleta \n'
---
14c15
< ('  * MERKUCJO — księcia \n', '  * MERKUCJO — krewny księcia \n')
> '  * MERKUCJO — krewny księcia \n'
---
15c16
< ('  * BENWOLIO — synowiec Montekiego \n', '  * BENWO